In [ ]:

# ----- Preparação para o modelo de classificação -----

# Dividir em conjuntos de treino e teste
X = aplicacoes_df.drop(['Aderente', 'CandidatoID', 'VagaID', 'Score_Total'], axis=1)
y = aplicacoes_df['Aderente']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Preprocessamento
numeric_features = ['Idade', 'Anos_Experiencia', 'Experiencia_Minima_Vaga', 
                    'Match_Habilidades', 'Match_Soft_Skills', 'Match_Experiencia', 
                    'Match_Salario', 'Match_Area']
categorical_features = ['Nivel_Formacao', 'Nivel_Vaga']

numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

# Criar e treinar o modelo
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('classifier', RandomForestClassifier(n_estimators=100, random_state=42))
])

# Treinar o modelo
model.fit(X_train, y_train)

# Avaliar o modelo
y_pred = model.predict(X_test)
y_prob = model.predict_proba(X_test)[:, 1]  # Probabilidade da classe positiva

# Métricas de avaliação
print("\n----- Avaliação do Modelo -----")
print(f"Acurácia: {accuracy_score(y_test, y_pred):.4f}")
print("\nRelatório de Classificação:")
print(classification_report(y_test, y_pred))

# Matriz de confusão
plt.figure(figsize=(8, 6))
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.xlabel('Previsto')
plt.ylabel('Real')
plt.title('Matriz de Confusão')
plt.savefig('matriz_confusao.png')
plt.close()

# Importância das features
if hasattr(model[-1], 'feature_importances_'):
    # Obter nomes das colunas após transformação
    ohe_features = []
    if hasattr(model[0], 'transformers_'):
        for name, _, column_names in model[0].transformers_:
            if name == 'cat':
                for col in column_names:
                    categories = model[0].named_transformers_[name]['onehot'].categories_[0]
                    for cat in categories:
                        ohe_features.append(f"{col}_{cat}")
            else:
                ohe_features.extend(column_names)
    
    # Usar os nomes originais se não conseguir encontrar os transformados
    if not ohe_features:
        ohe_features = X.columns
    
    # Importância das features
    feature_importances = model[-1].feature_importances_
    
    # Plotar importância das features
    plt.figure(figsize=(12, 8))
    indices = np.argsort(feature_importances)[::-1]
    top_indices = indices[:min(len(indices), 10)]  # Top 10 features
    
    plt.bar(range(len(top_indices)), feature_importances[top_indices])
    plt.xticks(range(len(top_indices)), [ohe_features[i] if i < len(ohe_features) else f"feature_{i}" for i in top_indices], rotation=45, ha='right')
    plt.title('Top 10 Features mais Importantes')
    plt.tight_layout()
    plt.savefig('feature_importance.png')
    plt.close()

# Salvar o modelo para uso posterior
joblib.dump(model, 'modelo_aderencia_candidatos.joblib')
print("\nModelo salvo como 'modelo_aderencia_candidatos.joblib'")

# ----- Criação de uma função para utilizar o modelo treinado -----
def classificar_candidato_para_vaga(candidato_dict, vaga_dict, modelo):
    ...

# ----- Exemplo de uso do modelo -----
print("\n----- Exemplo de uso do modelo -----")
modelo_carregado = joblib.load('modelo_aderencia_candidatos.joblib')
candidato_teste = candidatos_df.sample(1).iloc[0].to_dict()
vaga_teste = vagas_df.sample(1).iloc[0].to_dict()
print(f"Candidato: {candidato_teste['Nome']} - {candidato_teste['Formacao']}")
print(f"Vaga: {vaga_teste['Titulo']} - {vaga_teste['Empresa']}")
resultado = classificar_candidato_para_vaga(candidato_teste, vaga_teste, modelo_carregado)
print(f"Resultado: {resultado}")

# ----- Função para recomendação de vagas para um candidato -----
def recomendar_vagas_para_candidato(candidato_dict, todas_vagas_df, modelo, top_n=5):
    ...
candidato_recomendacao = candidatos_df.sample(1).iloc[0].to_dict()
amostra_vagas = vagas_df.sample(100)
recomendacoes = recomendar_vagas_para_candidato(candidato_recomendacao, amostra_vagas, modelo_carregado, top_n=5)
print(recomendacoes)
